In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("7-cyber_attack_data.csv")

In [3]:
print(df.info())
print(df.shape)
print(df.columns)
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   src_packet_rate              1000 non-null   float64
 1   dst_packet_rate              1000 non-null   float64
 2   avg_payload_size             1000 non-null   float64
 3   connection_duration          1000 non-null   float64
 4   tcp_flag_count               1000 non-null   float64
 5   avg_interarrival_time        1000 non-null   float64
 6   failed_login_attempts        1000 non-null   float64
 7   unusual_port_activity_score  1000 non-null   float64
 8   session_entropy              1000 non-null   float64
 9   avg_response_delay           1000 non-null   float64
 10  attack_type                  1000 non-null   int64  
dtypes: float64(10), int64(1)
memory usage: 86.1 KB
None
(1000, 11)
Index(['src_packet_rate', 'dst_packet_rate', 'avg_payload_size',
       

In [4]:
df.head()

,src_packet_rate,dst_packet_rate,avg_payload_size,connection_duration,tcp_flag_count,avg_interarrival_time,failed_login_attempts,unusual_port_activity_score,session_entropy,avg_response_delay,attack_type
0,-1.286132,-0.648334,1.044115,-0.469715,0.789859,-0.083727,-1.647309,-1.316412,1.011910,-0.898063,2
1,-0.222224,2.083232,1.191114,-1.354527,-0.956992,1.696028,-1.070406,0.981403,-1.628798,1.377594,0
2,-0.431963,0.375745,-1.370334,0.819214,0.345243,1.389447,-1.904130,1.292602,0.925545,0.232705,0
3,-0.912633,0.986988,-0.690042,2.014628,-0.442260,0.590347,-1.819353,1.560938,0.823755,0.517762,0
4,-0.367056,1.667892,0.879172,2.214276,1.846338,-0.894047,1.543838,0.931103,-1.015210,1.061845,1


In [5]:
# Bu datasette bir sunucuya gelen istekleri analiz ederek saldırı var mı yok mu anlamak istiyoruz.
# Ve üç ader saldırı sınıfımız var:
# 0 --> Saldırı yok
# 1 --> DDoS
# 2 --> Port Scan
# Biz multi-class logistic regression modelini kullanarak bunları tahmin etmeye çalışacağız.

In [6]:
from sklearn.model_selection import train_test_split
X = df.drop("attack_type", axis=1)
y = df["attack_type"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=15)

In [7]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
score = accuracy_score(y_pred, y_test)
print("Score: ", score)
print(classification_report(y_pred, y_test))
print("confusion matrix: \n", confusion_matrix(y_pred, y_test))

Score:  0.7866666666666666
              precision    recall  f1-score   support

           0       0.78      0.91      0.84        90
           1       0.74      0.73      0.73        99
           2       0.84      0.74      0.78       111

    accuracy                           0.79       300
   macro avg       0.79      0.79      0.79       300
weighted avg       0.79      0.79      0.79       300

confusion matrix: 
 [[82  2  6]
 [17 72 10]
 [ 6 23 82]]


In [9]:
# Zaten Encode ve Scale edilmiş datasetimizi ayırdık, lojistik regresyon modelimizi eğittik ve tahminlerimizi yaptık.
# Ardından modelimizin başarımını ölçmek için metriklerini inceleye başladık.
# Modelimizin doğruluğu ve başarı metrikleri bekledğimizden düşük çıktı.
# Çünkü aslında multi-class sınıflandırma yapmak istiyoruz ve lojitik regresyon ile yapmak adına OvR ve OvA stratejisini kullanmadık.
# Yani modelimizi binary classification yöntemi ile eğittik. 
# Burada hiperparametre ayarlaması yapmayı ve metrikleri arttırmayı deneyebiliriz.

In [10]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
tuning_model = LogisticRegression()
penalty = ["l1", "l2", "elasticnet"]
c_values = [100, 10, 1, 0.1, 0.01]
solvers = ["lbfsg", "liblinear", "sag", "saga", "newton-cg", "newton-cholesky"]
params = dict(penalty=penalty, C=c_values, solver=solvers)
cv = StratifiedKFold()
grid = GridSearchCV(estimator=tuning_model, param_grid=params, cv=cv, scoring="accuracy", n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [100, 10, 1, 0.1, 0.01],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['lbfsg', 'liblinear', 'sag', 'saga',
                                    'newton-cg', 'newton-cholesky']},
             scoring='accuracy')

In [11]:
print(grid.best_estimator_)
print(grid.best_params_)
y_pred_grid = grid.predict(X_test)
score = accuracy_score(y_pred_grid, y_test)
print("Score: ", score)
print(classification_report(y_pred_grid, y_test))
print("confusion matrix: \n", confusion_matrix(y_pred_grid, y_test))

LogisticRegression(C=0.1, penalty='l1', solver='liblinear')
{'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
Score:  0.7866666666666666
              precision    recall  f1-score   support

           0       0.78      0.92      0.85        89
           1       0.71      0.75      0.73        92
           2       0.87      0.71      0.78       119

    accuracy                           0.79       300
   macro avg       0.79      0.80      0.79       300
weighted avg       0.79      0.79      0.79       300

confusion matrix: 
 [[82  2  5]
 [15 69  8]
 [ 8 26 85]]


In [12]:
# Hiperparametre ayarlaması sonucunda da modelimizin başarısı artmadı. 
# Bu durumda farklı yöntemler denememiz gerekebilir.

In [13]:
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
ovo_model = OneVsOneClassifier(estimator=LogisticRegression())
ovr_model = OneVsRestClassifier(estimator=LogisticRegression())

In [14]:
ovo_model.fit(X_train, y_train)
y_pred = ovo_model.predict(X_test)
score = accuracy_score(y_pred, y_test)
print("Score: ", score)
print(classification_report(y_pred, y_test))
print("confusion matrix: \n", confusion_matrix(y_pred, y_test))

Score:  0.7966666666666666
              precision    recall  f1-score   support

           0       0.80      0.92      0.86        91
           1       0.74      0.76      0.75        95
           2       0.85      0.73      0.78       114

    accuracy                           0.80       300
   macro avg       0.80      0.80      0.80       300
weighted avg       0.80      0.80      0.80       300

confusion matrix: 
 [[84  3  4]
 [12 72 11]
 [ 9 22 83]]


In [15]:
ovr_model.fit(X_train, y_train)
y_pred = ovr_model.predict(X_test)
score = accuracy_score(y_pred, y_test)
print("Score: ", score)
print(classification_report(y_pred, y_test))
print("confusion matrix: \n", confusion_matrix(y_pred, y_test))

Score:  0.7833333333333333
              precision    recall  f1-score   support

           0       0.79      0.91      0.85        91
           1       0.71      0.73      0.72        94
           2       0.85      0.72      0.78       115

    accuracy                           0.78       300
   macro avg       0.78      0.79      0.78       300
weighted avg       0.79      0.78      0.78       300

confusion matrix: 
 [[83  2  6]
 [16 69  9]
 [ 6 26 83]]


In [16]:
# Sklearn kütüphanesinin multiclass modülünden OvO ve OvR class'larını import ettik.
# Bunlar multi-classification yaparken kullandığımız iki tekniği uygulayan modellerdir.
# Bu modellerle çalışırken estimator parametresi olarak hangi sınıflandırma algoritmasını kullanacağımızı belirtiriz.
# Daha sonra hem OvO hem de OvR için sonuçlarımızı değerlendiririz.
# Burada OvO sonucumuz daha yüksek çıktı. Zaten genel olarak bunun daha yüksek olmasını bekleriz.
# Çünkü OvO ile daha dengeli modeller kurabiliriz ama büyük datasetlerde ve fazla sınıflarda hesaplamak maliyetli olur.